Compile and Submit a Pipeline

In [ ]:
#papermill_description=Import_Python_Libraries
import pathlib
import random
import datetime
import time
import collections

In [ ]:
#papermill_description=Determine_Directory_Paths_For_Compiled_Pipelines
Jannah_Config = provisioner["inventory"]["group_vars"]["all"]["Jannah"]
Work_Dir = provisioner["inventory"]["group_vars"]["all"]["Jannah"]['global']['ansible']['working_dir']
MOLECULE_EPHEMERAL_DIRECTORY = provisioner["env"]["MOLECULE_EPHEMERAL_DIRECTORY"]
pipelines_dir = f"{Work_Dir}/ansible/library/pipelines"


Run Kind Cluster Matrix

In [ ]:
#papermill_description=Generate_Log_File_Name
current_datetime = datetime.datetime.fromtimestamp(time.time()).isoformat().replace(":","-").replace(".","-")


In [ ]:
pipeline_full_path = f"{Work_Dir}/pythogorean-{current_datetime}.yaml"
print(pipeline_full_path)

In [ ]:
from kfp import dsl

@dsl.component
def square(x: float) -> float:
    return x ** 2

@dsl.component
def add(x: float, y: float) -> float:
    return x + y

@dsl.component
def square_root(x: float) -> float:
    return x ** .5

@dsl.pipeline(name='pythagorean-theorem-pipeline',
              description='Solve for the length of a hypotenuse of a triangle with sides length `a` and `b`.',
              display_name='Pythagorean pipeline.')
def pythagorean(a: float, b: float) -> float:
    a_sq_task = square(x=a)
    b_sq_task = square(x=b)
    sum_task = add(x=a_sq_task.output, y=b_sq_task.output)
    return square_root(x=sum_task.output).output


In [ ]:
import kfp
kfp.compiler.Compiler().compile(
    pipeline_func=pythagorean,
    package_path=pipeline_full_path)

In [ ]:
import kfp
client = kfp.Client(host="http://127.0.0.1:8080")
client.list_experiments()

In [ ]:
run = client.create_run_from_pipeline_package(
    pipeline_full_path,
    arguments={
       'a' : 1.2, 'b' : 1.2,
    },
)

In [ ]:
pipeline_full_path = f"{Work_Dir}/boot-pipeline-{current_datetime}.yaml"
print(pipeline_full_path)
